In [ ]:
def sd(array):
    import numpy as np
    array=np.array(array)
    result=np.std(array)
    return result

def dnorm(x,mean=0,sd =1,log=False):
    from scipy.stats import norm
    if log==False:
        result=norm.pdf(x,loc=mean,scale=sd)
    elif log==True:
        result=norm.logpdf(x,loc=mean,scale=sd)
    return result

def dexp(x,loc=0,scale=1,log=False):
    from scipy.stats import expon
    if log==False:
        result = expon.pdf(x,loc=loc,scale=scale)
    elif log==True:
        result = expon.logpdf(x,loc=loc,scale=scale)
    return result

In [ ]:
import numpy as np
import pandas as pd

data = pd.read_csv("WaffleDivorce.csv",sep=';')

data_A = (data.MedianAgeMarriage - np.mean(data.MedianAgeMarriage) ) / np.std(data.MedianAgeMarriage)
data_D = (data.Divorce - data.Divorce.mean() ) / data.Divorce.std()
data_M = (data.Marriage - data.Marriage.mean() ) / data.Marriage.std()

In [ ]:
sd(data.MedianAgeMarriage)

In [ ]:
sd(data.Divorce)

In [ ]:
import pymc3 as pm
import arviz as az
import matplotlib.pyplot as plt

with pm.Model() as m5_1:
    a = pm.Normal("a", 0.0, 0.2)
    bA = pm.Normal("bA", 0.0, 0.5)
    mu = a + bA * data_A
    sigma = pm.Exponential("sigma", 1.0)
    obs = pm.Normal("Divorce", mu=mu, sigma=sigma, observed=data_D)
    prior_checks = pm.sample_prior_predictive(samples=50, random_seed=10)

In [ ]:
_, ax = plt.subplots()

x = np.linspace(-2, 2, 100)

for a, b in zip(prior_checks["a"], prior_checks["bA"]):
    y = a + b * x
    ax.plot(x, y, c="k", alpha=0.25)

ax.set_xlabel("Median Age of Marriage 'std'")
ax.set_ylabel("Divorce Rate 'std'")
ax.set_title("m5.1")

In [ ]:
with pm.Model() as m5_2:
    a = pm.Normal("a", 0.0, 0.2)
    bM = pm.Normal("bM", 0.0, 0.5)
    mu = a + bM * data_M
    sigma = pm.Exponential("sigma", 1.0)
    obs = pm.Normal("Divorce", mu=mu, sigma=sigma, observed=data_D)
    prior_checks = pm.sample_prior_predictive(samples=50, random_seed=10)

In [ ]:
_, ax = plt.subplots()

x = np.linspace(-2, 2, 50)

for a, b in zip(prior_checks["a"], prior_checks["bM"]):
    y = a + b * x
    ax.plot(x, y, c="k", alpha=0.25)

ax.set_xlabel("Marriage Rate 'std'")
ax.set_ylabel("Divorce Rate 'std'")
ax.set_title("m5.2")

In [ ]:
with pm.Model() as m5_3:
    a = pm.Normal("a", 0.0, 0.2)
    bM = pm.Normal("bM", 0.0, 0.5)
    bA = pm.Normal("bA", 0.0, 0.5)
    mu = a + bM * data_M + bA * data_A
    sigma = pm.Exponential("sigma", 1.0)
    obs = pm.Normal("Divorce", mu=mu, sigma=sigma, observed=data_D)
    prior_checks = pm.sample_prior_predictive(samples=50, random_seed=10)
    trace = pm.sample(1000, tune=2000, random_seed=10)

In [ ]:
func_dict = {
     "mean":  np.mean,
     "sd":    np.std,
     "5.5%":  lambda x: np.percentile(x, 5.5),
     "94.5%": lambda x: np.percentile(x, 94.5),
 }

In [ ]:
pm.summary(trace, var_names=["a", "bM", "bA", "sigma"],stat_funcs=func_dict,round_to=2,extend=False)

In [ ]:
data = pd.read_csv("milk.csv",sep=';').dropna()
data.rename(columns=lambda x: x.replace(".","_"),inplace=True)
data.head()

In [ ]:
data_K = (data.kcal_per_g - data.kcal_per_g.mean() ) / data.kcal_per_g.std()
data_N = (data.neocortex_perc - data.neocortex_perc.mean() ) / data.neocortex_perc.std()
data_M = (np.log(data.mass) - np.mean(np.log(data.mass)) ) / np.std(np.log(data.mass))

In [ ]:
with pm.Model() as m5_5_draft:
    a = pm.Normal("a", 0.0, 1.0)
    bN = pm.Normal("bN", 0.0, 1.0)
    mu = a + bN * data_N
    sigma = pm.Exponential("sigma", 1.0)
    obs = pm.Normal("kcal_per_g", mu=mu, sigma=sigma, observed=data_K)
    prior_checks = pm.sample_prior_predictive(samples=50, random_seed=10)

In [ ]:
_, ax = plt.subplots()

x = np.linspace(-2, 2, 50)

for a, b in zip(prior_checks["a"], prior_checks["bN"]):
    y = a + b * x
    ax.plot(x, y, c="k", alpha=0.25)

ax.set_xlabel("Neocortex Mass 'std'")
ax.set_ylabel("Kilocalories of Energy per Gram 'std'")
ax.set_title("m5.2")